In [313]:
import sqlite3
import pandas as pd
from surprise import SVD, accuracy
from surprise import Reader, Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
connection = sqlite3.connect('pubg_result.db')

In [314]:
# 에란겔 모델
data = pd.read_sql("SELECT * FROM PUBG;",connection)
data_erangel = data[data['map'] == 'Erangel (Remastered)']
data_erangel = data_erangel[data_erangel['rank'] != 'None']
data_to_erangel = data_erangel[['zone','damage_causer_name','rank']]
erangel_rating = data_erangel.pivot_table('rank', index = '_id', columns='damage_causer_name').fillna(0)

In [315]:
no_sup_item_ids = [] # 보급제외 무기
sup_item_ids = [] # 보급포함 무기
for i in erangel_rating.columns:
    no_sup_item_ids.append(i)
no_sup_item_ids.remove('WeapGroza_C')
no_sup_item_ids.remove('WeapMk14_C')
no_sup_item_ids.remove('WeapAUG_C')
no_sup_item_ids.remove('WeapM249_C')
no_sup_item_ids.remove('WeapAWM_C')
for i in erangel_rating.columns:
    sup_item_ids.append(i)

In [316]:
reader = Reader(rating_scale= (1, 95))
data = Dataset.load_from_df(df=data_to_erangel, reader=reader)
train, test = train_test_split(data, test_size=.25)
model_erangel = SVD(n_factors=100, n_epochs=20, random_state=123)
model_erangel.fit(train) # model 생성

In [317]:
def for_recommend(ml_zone,ml_weapons,model):
    zone = ml_zone # 선택지역
    actual_rating = 0
    a = 95
    for ml_weapon in ml_weapons :
        result = model.predict(zone, ml_weapon, actual_rating)
        if result.est < a:
            weapon_result = result
            a = result.est
    return weapon_result

In [318]:
recommend = for_recommend('school',no_sup_item_ids,model_erangel)
recommend.iid

'WeapP90_C'

In [319]:
predict_erangel = model_erangel.test(test)
accuracy.mae(predict_erangel)
accuracy.rmse(predict_erangel)

MAE:  12.8121
RMSE: 15.3939


15.393867814187223

In [320]:
# 미라마 모델
data = pd.read_sql("SELECT * FROM PUBG;",connection)
data_miramar = data[data['map'] == 'Miramar']
data_miramar = data_miramar[data_miramar['rank'] != 'None']
data_to_miramar = data_miramar[['zone','damage_causer_name','rank']]
miramar_rating = data_miramar.pivot_table('rank', index = '_id', columns='damage_causer_name').fillna(0)

In [321]:
reader = Reader(rating_scale= (1, 95))
data = Dataset.load_from_df(df=data_to_miramar, reader=reader)
train, test = train_test_split(data, test_size=.25)
model_miramar = SVD(n_factors=100, n_epochs=20, random_state=123)
model_miramar.fit(train) # model 생성

In [322]:
recommend = for_recommend('pecade',no_sup_item_ids,model_miramar)
recommend.iid

'WeapP90_C'

In [323]:
predict_miramar = model_miramar.test(test)
accuracy.mae(predict_miramar)
accuracy.rmse(predict_miramar)

MAE:  11.9698
RMSE: 14.6603


14.660337376374855

In [324]:
# 태이고 모델
data = pd.read_sql("SELECT * FROM PUBG;",connection)
data_teigo = data[data['map'] == 'Tiger_Main']
data_teigo = data_teigo[data_teigo['rank'] != 'None']
data_to_teigo = data_teigo[['zone','damage_causer_name','rank']]
teigo_rating = data_teigo.pivot_table('rank', index = '_id', columns='damage_causer_name').fillna(0)

In [325]:
reader = Reader(rating_scale= (1, 95))
data = Dataset.load_from_df(df=data_to_teigo, reader=reader)
train, test = train_test_split(data, test_size=.25)
model_teigo = SVD(n_factors=100, n_epochs=20, random_state=123)
model_teigo.fit(train) # model 생성

In [326]:
recommend = for_recommend('palace',no_sup_item_ids,model_teigo)
recommend.iid

'WeapMG3_C'

In [327]:
predict_teigo = model_teigo.test(test)
accuracy.mae(predict_teigo)
accuracy.rmse(predict_teigo)

MAE:  13.9085
RMSE: 16.9773


16.977338062962406

In [328]:
import pickle

with open('model_erangel.pkl','wb') as pickle_file:
    pickle.dump(model_erangel, pickle_file)

with open('model_miramar.pkl','wb') as pickle_file:
    pickle.dump(model_miramar, pickle_file)

with open('model_teigo.pkl','wb') as pickle_file:
    pickle.dump(model_teigo, pickle_file)

In [334]:
print(sup_item_ids)

['BP_BRDM_C', 'BP_CoupeRB_C', 'BP_FireEffectController_C', 'BP_FireEffectController_JerryCan_C', 'BP_JerryCanFireDebuff_C', 'BP_MolotovFireDebuff_C', 'BP_Motorbike_04_C', 'BP_Motorbike_04_SideCar_C', 'BP_Motorbike_Solitario_C', 'Boat_PG117_C', 'Buggy_A_01_C', 'Buggy_A_02_C', 'Buggy_A_03_C', 'Dacia_A_01_v2_C', 'Dacia_A_02_v2_C', 'Dacia_A_03_v2_C', 'Dacia_A_03_v2_Esports_C', 'Dacia_A_04_v2_C', 'DroppedItemGroup', 'None', 'PlayerFemale_A_C', 'PlayerMale_A_C', 'ProjC4_C', 'ProjGrenade_C', 'ProjMolotov_C', 'Uaz_A_01_C', 'Uaz_B_01_C', 'Uaz_B_01_esports_C', 'Uaz_C_01_C', 'WeapACE32_C', 'WeapAK47_C', 'WeapAUG_C', 'WeapAWM_C', 'WeapBerreta686_C', 'WeapBerylM762_C', 'WeapBizonPP19_C', 'WeapCowbarProjectile_C', 'WeapCowbar_C', 'WeapCrossbow_1_C', 'WeapDP12_C', 'WeapDP28_C', 'WeapDesertEagle_C', 'WeapFNFal_C', 'WeapG18_C', 'WeapGroza_C', 'WeapHK416_C', 'WeapKar98k_C', 'WeapM16A4_C', 'WeapM1911_C', 'WeapM249_C', 'WeapM24_C', 'WeapM9_C', 'WeapMG3_C', 'WeapMacheteProjectile_C', 'WeapMachete_C', 'Weap